In [2]:
import yfinance as yf
import pandas as pd
import pandas_ta as pdta
import matplotlib.pyplot as plt
import numpy as np
import ta

In [3]:
bseTicker = pd.read_html('https://en.wikipedia.org/wiki/BSE_SENSEX')[1].Symbol.to_list()

In [4]:
bseTicker

['ASIANPAINT.BO',
 'AXISBANK.BO',
 'BAJAJ-AUTO.BO',
 'BAJFINANCE.BO',
 'BAJAJFINSV.BO',
 'BHARTIARTL.BO',
 'DRREDDY.BO',
 'HCLTECH.BO',
 'HDFC.BO',
 'HDFCBANK.BO',
 'HINDUNILVR.BO',
 'ICICIBANK.BO',
 'INDUSINDBK.BO',
 'INFY.BO',
 'ITC.BO',
 'KOTAKBANK.BO',
 'LT.BO',
 'M&M.BO',
 'MARUTI.BO',
 'NESTLEIND.BO',
 'NTPC.BO',
 'ONGC.BO',
 'POWERGRID.BO',
 'RELIANCE.BO',
 'SBIN.BO',
 'SUNPHARMA.BO',
 'TCS.BO',
 'TECHM.BO',
 'TITAN.BO',
 'ULTRACEMCO.BO']

In [ ]:
data = []

data.append(yf.download(bseTicker[0],start='2010-01-01').reset_index())
bseTicker[0]
#for ticker in bseTicker:
#    data.append(yf.download(ticker).reset_index())

In [4]:
def plotChart(data,bseTicker):
    pricex = data['Date']
    pricey = data['Adj Close']
    smax = data['Date']
    smay = data['SMA200']
    
    superX = data['Date']
    superY = data['st']
    
    plt.rcParams["figure.figsize"] = (20, 10)
    plt.plot(pricex,pricey)
    plt.plot(smax,smay)
    plt.plot(superX,superY)
    plt.title(bseTicker)
    plt.show()

In [5]:
data[0]['SMA200']=ta.trend.sma_indicator(data[0]['Adj Close'], window=200, fillna=False)
data[0] = data[0].dropna()



In [6]:
data[0]

,Date,Open,High,Low,Close,Adj Close,Volume,SMA200
199,2010-10-19,269.799988,269.799988,263.000000,263.690002,240.607239,61410,205.827621
200,2010-10-20,262.000000,263.700012,260.000000,260.339996,237.550430,168840,206.205285
201,2010-10-21,264.799988,264.799988,261.500000,262.885010,239.872650,41170,206.594220
202,2010-10-22,263.515015,264.494995,261.515015,262.089996,239.147293,271050,206.984056
203,2010-10-25,264.500000,264.500000,261.299988,262.160004,239.211166,42820,207.375975
...,...,...,...,...,...,...,...,...
2923,2021-11-12,3080.000000,3130.000000,3076.050049,3122.899902,3122.899902,43214,2852.455446
2924,2021-11-15,3126.000000,3179.949951,3126.000000,3167.899902,3167.899902,59064,2854.887717
2925,2021-11-16,3167.000000,3209.399902,3120.149902,3152.050049,3152.050049,30998,2857.157501
2926,2021-11-17,3153.000000,3239.000000,3153.000000,3229.800049,3229.800049,138845,2860.401938


In [16]:
def get_supertrend(high, low, close, lookback, multiplier):
    
    # ATR
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.ewm(lookback).mean()
    
    # H/L AVG AND BASIC UPPER & LOWER BAND
    
    hl_avg = (high + low) / 2
    upper_band = (hl_avg + multiplier * atr).dropna()
    lower_band = (hl_avg - multiplier * atr).dropna()
    
   
    # FINAL UPPER BAND
    
    final_bands = pd.DataFrame(columns = ['upper', 'lower'])
    final_bands.iloc[:,0] = [x for x in upper_band - upper_band]
    final_bands.iloc[:,1] = final_bands.iloc[:,0]
    
    for i in range(len(final_bands)):
        print
        if i == 0:
            final_bands.iloc[i,0] = 0
        else:
            if (upper_band[i] < final_bands.iloc[i-1,0]) | (close[i-1] > final_bands.iloc[i-1,0]):
                final_bands.iloc[i,0] = upper_band[i]
            else:
                final_bands.iloc[i,0] = final_bands.iloc[i-1,0]
    
    # FINAL LOWER BAND
    
    for i in range(len(final_bands)):
        if i == 0:
            final_bands.iloc[i, 1] = 0
        else:
            if (lower_band[i] > final_bands.iloc[i-1,1]) | (close[i-1] < final_bands.iloc[i-1,1]):
                final_bands.iloc[i,1] = lower_band[i]
            else:
                final_bands.iloc[i,1] = final_bands.iloc[i-1,1]
    
    # SUPERTREND
    
    supertrend = pd.DataFrame(columns = [f'supertrend_{lookback}'])
    supertrend.iloc[:,0] = [x for x in final_bands['upper'] - final_bands['upper']]
    
    for i in range(len(supertrend)):
        if i == 0:
            supertrend.iloc[i, 0] = 0
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 0] and close[i] < final_bands.iloc[i, 0]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 0]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 0] and close[i] > final_bands.iloc[i, 0]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 1]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 1] and close[i] > final_bands.iloc[i, 1]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 1]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 1] and close[i] < final_bands.iloc[i, 1]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 0]
    
    supertrend = supertrend.set_index(upper_band.index)
    supertrend = supertrend.dropna()[1:]
    
    # ST UPTREND/DOWNTREND
    
    upt = []
    dt = []
    close = close.iloc[len(close) - len(supertrend):]

    for i in range(len(supertrend)):
        if close[i] > supertrend.iloc[i, 0]:
            upt.append(supertrend.iloc[i, 0])
            dt.append(np.nan)
        elif close[i] < supertrend.iloc[i, 0]:
            upt.append(np.nan)
            dt.append(supertrend.iloc[i, 0])
        else:
            upt.append(np.nan)
            dt.append(np.nan)
            
    st, upt, dt = pd.Series(supertrend.iloc[:, 0]), pd.Series(upt), pd.Series(dt)
    upt.index, dt.index = supertrend.index, supertrend.index
    
    return st, upt, dt


In [17]:
ticker = data[0]
get_supertrend(ticker['High'], ticker['Low'], ticker['Close'], 3, 2.75)

199      285.099960
200      275.678582
201      276.302419
202      274.344409
203      273.407031
           ...     
2923    3287.335104
2924    3330.429444
2925    3359.225129
2926    3401.615761
2927    3422.883696
Length: 2729, dtype: float64